In [ ]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets
from tqdm.notebook import tqdm

import sys
sys.path.append('/home/jumper/projects/ballsbot/python/lib')

import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [ ]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import revert_transformation_to_cloud

In [ ]:
only_nearby_meters = 4

augmented_image = widgets.Image(format='jpeg', width=1000, height=1000)
sparse_image = widgets.Image(format='jpeg', width=1000, height=1000)
display(widgets.HBox([augmented_image, sparse_image]))

In [ ]:
new_format = True
with open('track_1-16-0_02.json', 'r') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [ ]:
calibration = {
    'angle_fix': 3.0660432755299887,
    'fl_x': 0.3443406823910605,
    'fl_y': 0.15713610488972193,
    'rr_x': -0.07267960972661196,
    'rr_y': -0.15285037016562764,
}
self_position = calibration_to_xywh(calibration)
augmented_painter = BotPoseDrawing(augmented_image)
sparse_painter = BotPoseDrawing(sparse_image)

In [ ]:
# ts = None
# for frame_index, track_frame in enumerate(tqdm(track)):
#     if not track_frame['tracker']['current_pose']:
#         continue
#     ts = keep_rps(ts, fps=1)
#     pose = {'ts': track_frame['tracker']['current_pose']['ts'], 'x': 0., 'y': 0., 'teta': 0.}

#     image_text = json.dumps(track_frame['direction'], sort_keys=True)
#     if track_frame['lidar']['augmented_points']:
#         augmented_painter.update_image(
#             [pose],
#             track_frame['lidar']['augmented_points'],
#             self_position,
#             image_text=image_text,
#             only_nearby_meters=only_nearby_meters
#         )
#     if track_frame['lidar']['sparse_points']:
#         if track_frame['tracker']['free_tile_centers']:
#             cells = [track_frame['tracker']['target_point']] + track_frame['tracker']['free_tile_centers']
#             transformation = (track_frame['tracker']['current_pose']['x'], track_frame['tracker']['current_pose']['y'], track_frame['tracker']['current_pose']['teta'])
#             transformed_cells = revert_transformation_to_cloud(cells, transformation)
#             free_cells = transformed_cells[1:]
#             target_point = transformed_cells[0]
#         else:
#             free_cells = None
#             target_point = None
        
#         sparse_painter.update_image(
#             [pose],
#             track_frame['lidar']['sparse_points'],
#             self_position,
#             free_cells=free_cells,
#             target_point=target_point,
#             image_text=image_text,
#             only_nearby_meters=only_nearby_meters
#         )

#     clear_output(wait=True)
#     print(frame_index)
#     print(json.dumps(track_frame['directions_weights'], indent=4, sort_keys=True))
# #     if frame_index> 60 and track_frame['directions_weights'] and track_frame['directions_weights']['(0.0,-1.0)'] == 0.:
# #         break

In [ ]:
# for it in track[81]['lidar']['sparse_points']:
#     print(f'{{{it[0]}, {it[1]}}},')

In [ ]:
# track[81]['lidar']['grid_pose']

In [ ]:
def subtract(ts1, ts2):
    if ts1 is None:
        return None
    else:
        return ts1 - ts2

lines = {
    'index': [],
    'odometry': [],
    'imu': [],
    'pose': [],
    'lidar': [],
    'sparse_lidar': [],
    "ls-front-center": [],
    "ls-front-left": [],
    "ls-front-right": [],
    "ls-rear-center": [],
    "ls-rear-left": [],
    "ls-rear-right": [],
}
for frame_index, track_frame in enumerate(tqdm(track)):
    real_time = track_frame['ts']
    if real_time is None:
        continue
    lines['index'].append(frame_index)
    lines['odometry'].append(subtract((track_frame['tracker']['current_pose'] or {}).get('odometry_ts'), real_time))
    lines['imu'].append(subtract((track_frame['tracker']['current_pose'] or {}).get('imu_ts'), real_time))
    lines['pose'].append(subtract((track_frame['tracker']['current_pose'] or {}).get('self_ts'), real_time))
    lines['lidar'].append(subtract(track_frame['lidar'].get('points_ts'), real_time))
    lines['sparse_lidar'].append(subtract(track_frame['lidar'].get('update_grid_ts'), real_time))
    for ls_name in ('front-center', 'front-left', 'front-right', 'rear-center', 'rear-left', 'rear-right'):
        lines["ls-"+ls_name].append(subtract(((track_frame['lidar']['distances'] or {}).get(ls_name) or {}).get('ts'), real_time))

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 8))
for line_name, y_points in lines.items():
    if line_name == 'index' or line_name.startswith('ls-'):
        continue
    plt.plot(lines['index'], y_points, label=line_name)

plt.grid(True)
plt.legend()
plt.show()